In [2]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.3 MB/s eta 0:00:00


In [3]:
import requests
from docx import Document

In [4]:
from google.colab import userdata

ttl_subscription_key = userdata.get('ttl_subscription_key')
ttl_endpoint = userdata.get('ttl_endpoint')
ttl_location = userdata.get('ttl_location')

In [14]:
import os

destination_language = "pt-br"

def translator_text(text, target_language):
  path = '/translate'
  constructed_url = ttl_endpoint + path

  params = {
      'api-version': '3.0',
      'from': 'en',
      'to': [target_language]
  }
  headers = {
      'Ocp-Apim-Subscription-Key': ttl_subscription_key,
      'Ocp-Apim-Subscription-Region': ttl_location,
      'Content-type': 'application/json',
      'X-ClientTraceId': str(os.urandom(16))
  }
  body = [{"text": text}]
  response = requests.post(constructed_url, params=params, headers=headers, json=body)
  data = response.json()
  return data[0]["translations"][0]["text"]


In [15]:
translator_text("I will not say, do not weep, because not all tears are evil.", destination_language)

'Eu não direi, não choreis, porque nem todas as lágrimas são más.'

In [17]:
def translate_document(path):
  document = Document(path)
  full_text = []
  for paragraph in document.paragraphs:
    full_text.append(translator_text(paragraph.text, destination_language))

  translated_document = Document()
  for line in full_text:
    translated_document.add_paragraph(line)

  path_translated = path.replace(".docx", f"_{destination_language}.docx")
  translated_document.save(path_translated)
  return path_translated

In [18]:
input_file = "/content/Mirrors.docx"
translate_document(input_file)

'/content/Mirrors_pt-br.docx'

In [19]:
!pip install openai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.4 MB/s eta 0:00:00


In [20]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
  response = requests.get(url)

  if response.status_code != 200:
    raise Exception(f"Failed to fetch the URL. Status code: {response.status_code}")

  soup = BeautifulSoup(response.text, 'html.parser')
  for script in soup(["script", "style"]):
    script.decompose()
  text = soup.get_text(separator=" ")
  #Limpar texto
  lines = (line.strip() for line in text.splitlines())
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  text = '\n'.join(chunk for chunk in chunks if chunk)
  return text

In [21]:
from google.colab import userdata

azure_open_ai_key = userdata.get('azure_open_ai_key')

In [24]:
from langchain_openai import AzureChatOpenAI

client = AzureChatOpenAI(
    azure_endpoint="https://oai-dio-bootcamp-dev-eastus-777.openai.azure.com/",
    api_key=azure_open_ai_key,
    api_version="2024-12-01-preview",
    deployment_name="gpt-4o-mini",
    max_retries=0
)

def translate_article(text, lang):
  messages = [
      ("system", "Você atua como tradutor de textos"),
      ("user", f"Traduza o {text} para o idioma {lang} e responda em markdown.")
  ]

  response = client.invoke(messages)
  return response.content

In [25]:
url = "https://cloud.google.com/learn/what-is-artificial-intelligence"
text = extract_text_from_url(url)
article = translate_article(text, "pt-br")

In [26]:
article

'# O Que É Inteligência Artificial (IA)? | Conteúdos da Página do Google Cloud\n\n## O que é Inteligência Artificial?\n\nA inteligência artificial (IA) é um conjunto de tecnologias que permite que computadores realizem uma variedade de funções avançadas, incluindo a capacidade de **ver**, **entender** e **traduzir** a linguagem falada e escrita, **analisar dados**, fazer **recomendações**, entre outras. A IA é a espinha dorsal da inovação na computação moderna, desbloqueando valor para indivíduos e empresas. Por exemplo, o reconhecimento óptico de caracteres (OCR) utiliza a IA para extrair texto e dados de imagens e documentos, transformando conteúdo não estruturado em dados estruturados prontos para o negócio, e revelando insights valiosos.\n\n### Pronto para começar?\n\nNovos clientes recebem $300 em créditos gratuitos para gastar no Google Cloud. **Comece gratuitamente!**\n\n## Introdução à IA Generativa\n\n### Definição de Inteligência Artificial\n\nA inteligência artificial é um c